In [1]:
import os
import pandas as pd
import torch
from tqdm import tqdm

In [2]:
#Use the data extracted in part 1. Replace 'dataset5/asl-signs/train_landmark_files' below with its path
dataset = []
for dirname, _, filenames in os.walk('train_landmark_files_preprocessed'):
    for filename in filenames:
        if filename.endswith('.parquet'):
            dataset.append(os.path.join(dirname, filename))
dataset

['train_landmark_files_preprocessed/16069/100015657.parquet',
 'train_landmark_files_preprocessed/16069/1002113535.parquet',
 'train_landmark_files_preprocessed/16069/10042041.parquet',
 'train_landmark_files_preprocessed/16069/1004211348.parquet',
 'train_landmark_files_preprocessed/16069/100438640.parquet',
 'train_landmark_files_preprocessed/16069/1005009451.parquet',
 'train_landmark_files_preprocessed/16069/1005223850.parquet',
 'train_landmark_files_preprocessed/16069/1005492440.parquet',
 'train_landmark_files_preprocessed/16069/1006058791.parquet',
 'train_landmark_files_preprocessed/16069/1006443357.parquet',
 'train_landmark_files_preprocessed/16069/1007073382.parquet',
 'train_landmark_files_preprocessed/16069/1007127288.parquet',
 'train_landmark_files_preprocessed/16069/1007343357.parquet',
 'train_landmark_files_preprocessed/16069/1007589714.parquet',
 'train_landmark_files_preprocessed/16069/1008185618.parquet',
 'train_landmark_files_preprocessed/16069/1009615872.parque

In [2]:
input_dir = 'train_landmark_files_preprocessed' # adjust as needed
output_dir = "train_landmark_files_normalized"
os.makedirs(output_dir, exist_ok=True)

In [3]:
# Accumulate stats
all_means, all_stds, all_mins, all_maxs = [], [], [], []

In [4]:
file_list = []
for root, _, files in os.walk(input_dir):
    for f in files:
        if f.endswith(".parquet"):
            file_list.append(os.path.join(root, f))

In [5]:
# First pass: compute global stats
for file in tqdm(file_list, desc="Computing stats"):
    df = pd.read_parquet(file)
    tensor = torch.tensor(df.values, dtype=torch.float32).cuda()

    all_means.append(tensor.mean(dim=0))
    all_stds.append(tensor.std(dim=0))
    all_mins.append(tensor.min(dim=0)[0])
    all_maxs.append(tensor.max(dim=0)[0])

global_mean = torch.stack(all_means).mean(dim=0)
global_std = torch.stack(all_stds).mean(dim=0)


Computing stats: 100%|██████████| 94477/94477 [28:55<00:00, 54.44it/s]


In [6]:
# Second pass: normalize and save
for file in tqdm(file_list, desc="Normalizing files"):
    df = pd.read_parquet(file)
    tensor = torch.tensor(df.values, dtype=torch.float32).cuda()

    normed_tensor = (tensor - global_mean) / global_std
    normed_df = pd.DataFrame(normed_tensor.cpu().numpy(), columns=df.columns)

    # Preserve directory structure
    rel_path = os.path.relpath(file, input_dir)
    save_path = os.path.join(output_dir, rel_path)
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    normed_df.to_parquet(save_path)

Normalizing files: 100%|██████████| 94477/94477 [3:17:06<00:00,  7.99it/s]  


In [3]:
len(dataset)

94477

In [16]:
dataset1 = []
for dirname, _, filenames in os.walk('train_landmark_files_normalized'):
    for filename in filenames:
        if filename.endswith('.parquet'):
            dataset6.append(os.path.join(dirname, filename))
len(dataset1)

94477